In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

In [2]:
import torch
import torch.nn as nn
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [1]:
import numpy as np
import pandas as pd

In [4]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None) 

In [5]:
train = pd.read_pickle("../bert/p1_codemix_flat.pkl")

In [6]:
train

,tweet_id,context,text,label
0,1397101600460529665,,Countries which have Banned Twitter\n\n🇨🇳 Chin...,HOF
1,1397101827116703744,Countries which have Banned Twitter\n\n🇨🇳 Chin...,@srivatsayb #IStandWithTwitterIndia\nOur belov...,NONE
2,1397101939674869763,Countries which have Banned Twitter\n\n🇨🇳 Chin...,@srivatsayb Srivatsa in that case just say onc...,HOF
3,1397102700173488133,Countries which have Banned Twitter\n\n🇨🇳 Chin...,@Lawyer_Sandeep @srivatsayb That's a fact\nThe...,HOF
4,1397102906004754433,Countries which have Banned Twitter\n\n🇨🇳 Chin...,@Lawyer_Sandeep @srivatsayb Corona is from chi...,HOF
...,...,...,...,...
2814,1385540853586751492,So what if Delhi CM highlighting acute oxygen ...,@MahuaMoitra Tu तलवे चाट,HOF
2815,1385540878970679299,So what if Delhi CM highlighting acute oxygen ...,@MahuaMoitra Your concern on other state delay...,NONE
2816,1385541203458854916,So what if Delhi CM highlighting acute oxygen ...,@MahuaMoitra telecasting without consent? Also...,NONE
2817,1385541327602036739,So what if Delhi CM highlighting acute oxygen ...,@MahuaMoitra @VidyaKrishnan Out of syllabus qu...,NONE


In [7]:
!pip install transformers --quiet

     |████████████████████████████████| 3.5 MB 12.4 MB/s 
     |████████████████████████████████| 596 kB 42.3 MB/s 
     |████████████████████████████████| 6.8 MB 34.3 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 895 kB 48.1 MB/s 


In [8]:
from transformers import pipeline
class XLMR:
    def __init__(self):
        self.model_name = 'xlmr'
        self.nlp = pipeline(task ="feature-extraction", model = 'xlm-roberta-base', tokenizer='xlm-roberta-base', framework='pt', device=0)

    def GetFeatures(self, sentences=None):
        if self.model_name == 'xlmr':
            features = self.nlp(sentences, truncation=True)      
            return pd.DataFrame(features[0][0])

In [9]:
xlmr = XLMR()

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [ ]:
import pandas as pd


In [10]:
class CustomBERTModel(nn.Module):
    def __init__(self):
          super(CustomBERTModel, self).__init__()
          self.linear = nn.Linear(768,2)
          self.outputlayer = nn.Softmax(dim=1)
      
    def forward(self, row_embeddings=None):
        linear_output = self.linear(row_embeddings)
        outputs = self.outputlayer(linear_output)
        return outputs

In [11]:
model = CustomBERTModel()

In [12]:
learning_rate = 1e-4
epochs = 4
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

In [13]:
model = model.to(device)

In [14]:
from operator import add
from scipy import spatial

In [15]:
  TASK1 = {'HOF': 1, 'NONE': 0}
  train["label"]=train["label"].map(TASK1)
  lambda1=0.4
  lambda2=0.6

In [97]:
print("Training....")

for epoch in range(epochs):
    total_train_loss=0
    print("==========Epochs:{}===========".format(epoch)) 
    for context,text,label in zip(train["context"], train["text"], train["label"]):
      row_embeddings=[]
      f_embeddings=[]
      text_embeddings = xlmr.GetFeatures(text)
      context_embeddings=xlmr.GetFeatures(context)
      text_embeddings =  lambda2*text_embeddings
      context_embeddings =lambda1*context_embeddings
      final_embeddings = text_embeddings+context_embeddings
      
      for i in final_embeddings.values:
         f_embeddings.append(i[0])
    
      row_embeddings.append(f_embeddings) 
      row_embeddings=torch.Tensor(row_embeddings).to(device) 
      logits=model(row_embeddings)     
      one_hot = torch.zeros(1, 2)
      one_hot[0, label] = 1
      one_hot = one_hot.to(device)    
      loss = criterion(input = logits, target = one_hot)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      total_train_loss += loss
      
      torch.save(model.state_dict(), "rev_xlmr_model_adamw_b3.pth")
    print("Training Loss: {}".format((total_train_loss/len(train)).item()))

Training....
==========Epochs:0===========


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:978: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


Training Loss: 0.6860576868057251
==========Epochs:1===========
Training Loss: 0.6860576868057251
==========Epochs:2===========
Training Loss: 0.6860576868057251
==========Epochs:3===========
Training Loss: 0.6860576868057251


In [90]:
torch.save(model.state_dict(), "rev_xlmr_model_adamw_b3.pth")

In [91]:
model = CustomBERTModel()
model.load_state_dict(torch.load("rev_xlmr_model_adamw_b3.pth"))

<All keys matched successfully>

In [ ]:
# Testing Setup

In [92]:
test = pd.read_pickle("p1_codemix_flat_test.pkl")

In [94]:
test['label']=test['label'].map(TASK1)

In [96]:
from sklearn.metrics import accuracy_score, f1_score
print("Testing....")
predictions = []
labels = []
model.eval()
model=model.to(device)
with torch.no_grad():
  for  context, text, label in zip(test["context"], test["text"], test["label"]):
      row_embeddings=[]
      f_embeddings=[]
      text_embeddings = xlmr.GetFeatures(text)
      context_embeddings=xlmr.GetFeatures(context)
      text_embeddings =  lambda2*text_embeddings
      context_embeddings =lambda1*context_embeddings
      final_embeddings = text_embeddings+context_embeddings
      for i in final_embeddings.values:
         f_embeddings.append(i[0])
      row_embeddings.append(f_embeddings) 
      row_embeddings=torch.Tensor(row_embeddings).to(device) 
      logits=model(row_embeddings) 
      y_pred = torch.argmax(logits, dim=-1).item()
      predictions.append(y_pred)
      labels.append(label)

  print("Accuracy: {}".format(accuracy_score(predictions, labels)))
  print("F1 Score: {}".format(f1_score(predictions, labels)))

Testing....


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:978: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


Accuracy: 1.0
F1 Score: 1.0


In [ ]:
import pickle

with open('XLMR_1e-4.pkl', 'wb') as f:
  pickle.dump(predictions, f)